# Necessary packages webscrapping

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import os
import pandas as pd
import numpy as np

# FOR APPLYING SELENIUM:
import selenium # Python Selenium
from selenium import webdriver # for specifying webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import time # for stopping system for a while (in case of some delays when accessing a website,
            # also helpful in case of captchas)

from webdriver_manager.chrome import ChromeDriverManager # chromedriver for automatized access to Chrome
from webdriver_manager.firefox import GeckoDriverManager # geckodriver for automatized access to Firefox
from webdriver_manager.microsoft import EdgeChromiumDriverManager # msedgedriver for automatized access to Microsoft Edge

from selenium.webdriver.chrome.service import Service # needed since Selenium 4.10.0 see: https://github.com/SeleniumHQ/selenium/commit/9f5801c82fb3be3d5850707c46c3f8176e3ccd8e

import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager #webdriver for firefox
from selenium.webdriver.firefox.service import Service #firefox
import time
from bs4 import BeautifulSoup
from selenium.webdriver.firefox.options import Options #options for firefox
import re


In [2]:
#chromepath = ChromeDriverManager().install()
firefoxpath = GeckoDriverManager().install() #we will use firefox (since chrome didn't work for the author)
#edgepath = EdgeChromiumDriverManager().install()

#print(chromepath)
print(firefoxpath)
#print(edgepath)

C:\Users\Piotr\.wdm\drivers\geckodriver\win64\v0.35.0\geckodriver.exe


# Accessing the website content


In [3]:
%%capture 
####################REMOVE %%capture LINE IF YOU WANT TO SEE THE OUTPUT############################################
#we use %%capture because the output is too long for demonstrative purposes
#Let's determine our address page we are interested in scraping
url = "https://wykop.pl/hity/roku/strona/1"

# Making a GET request
response = requests.get(url)

# extract the text
html = response.text

html


In [4]:
%%capture 
####################REMOVE %%capture LINE IF YOU WANT TO SEE THE OUTPUT############################################
#we use %%capture because the output is too long for demonstrative purposes
#using beautiful soup to parse the html
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())
# we use the .prettify() function of the beautifulsoup4 so that the output doesn't look like the jumbled mess above

<!DOCTYPE html>
<html class="chrome desktop landscape" lang="pl-PL">
 <head>
  <meta charset="utf-8"/>
  <title>
   Hity (roku) - strona 1 :: Wykop.pl
  </title>
  <meta content="Wykop Sp. z o.o." name="Author"/>
  <meta content="pl_PL" property="og:locale"/>
  <meta content="website" property="og:type"/>
  <meta content="Wykop" property="og:site_name"/>
  <link href="/manifest.json" rel="manifest"/>
  <link href="/static/open-search.xml" rel="search" title="Wyszukiwarka Wykop.pl" type="application/opensearchdescription+xml"/>
  <meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/>
  <link href="/static/img/favicons/favicon-180.png" rel="apple-touch-icon"/>
  <link href="/static/img/favicons/favicon.png" rel="alternate icon" type="image/png"/>
  <link href="/static/img/favicons/favicon.svg" rel="icon" type="image/svg+xml"/>
  <style>
   body.ihmn .prerender {visibility: hidden;} body:not(.ihmn):has(section.prerender) section[class*=-layout]:not(.prerender) {d

While looking at the code by ourselves we can see that the upvotes are from the "dig" div class

# Getting the text from the website


In [5]:
soup.get_text()

'Hity (roku) - strona 1 :: Wykop.pl    Menu Wykop.pl Wykop.pl  Główna Wykopalisko314 Mikroblog Szukaj:        Zaloguj się Zarejestruj się        Wykop.pl  Ciekawostki   CiekawostkiRozrywka   RozrywkaSport   SportMotoryzacja   MotoryzacjaTechnologia   TechnologiaInformacje   InformacjeGospodarka   GospodarkaPodróże   Podróże   Dostosuj Wykop do siebie Utwórz Kategorię   Wybierz treści, które Cię interesują  i zapisz jako Własną kategorię.     Tryb dzienny         Hity roku Archiwum        11524 Wykop    \n                            Sebastian M. wyszedł na wolność\n                           Sebastianowi M. podejrzanemu o spowodowanie tragicznego wypadku na A1, w którym zginęła trzyosobowa rodzina, został uchylony areszt tymczasowy w Dubaju. Mężczyzna został zatrzymany w ubiegłym roku na terenie Zjednoczonych Emiratów Arabskich.    \n        bylemtukiedys12\n                            z\n                              tvn24.pl opublikowany: 02.02.2024, 09:22:25        \n            796\

In [6]:
html = requests.get("https://wykop.pl/hity/roku/strona/1")
html

<Response [200]>

In [7]:
html.status_code ==200

True

Response code 200 means that the request has succeeded

Looking up upvote class

In [8]:
soup.find(class_="dig") # "dig" is the class for upvotes

<div class="dig" data-v-5bb34f93=""><p data-v-5bb34f93=""><span data-v-5bb34f93="">11524</span></p> <button data-v-5bb34f93="">Wykop</button></div>

In [9]:
soup.find(class_="tag") # "tag" is the class for tags

<li class="tag" data-v-0e7a0051=""><span data-v-0e7a0051="">#</span><div class="tooltip-slot" data-v-0e7a0051="" data-v-d8ca489a=""><span data-v-d8ca489a=""><span style="display: none;"><section class="popper tooltip-slot tag-actions guest" data-v-d8ca489a=""><!-- --></section> </span> <a class="" data-v-0e7a0051="" href="/tag/majtczak">majtczak</a></span></div></li>

In [10]:
#we can see that comments refers to the number of comments
soup.find(class_="comments")

<li class="comments" data-v-0e7a0051=""><!--fragment#3980e396ec#head--><a class="comment-counter" data-v-c10ff764="" fragment="3980e396ec" href="/link/7353491/sebastian-m-wyszedl-na-wolnosc">
            796
        </a><!--fragment#3980e396ec#tail--></li>

In [11]:
#that's too vague - we want the "comment-counter" class
soup.find(class_="comment-counter")

<a class="comment-counter" data-v-c10ff764="" fragment="3980e396ec" href="/link/7353491/sebastian-m-wyszedl-na-wolnosc">
            796
        </a>

In [12]:
comment_counter=soup.find(class_="comment-counter")
comment_counter.attrs

{'data-v-c10ff764': '',
 'href': '/link/7353491/sebastian-m-wyszedl-na-wolnosc',
 'class': ['comment-counter'],
 'fragment': '3980e396ec'}

We have the css selector, data-v-c10ff764. However, we can't just put it in there and be happy because there is a chance the code won't work in the future. After performing webscraping on 3 different pages after some time, the **css selectors changed over time.** Therefore we should either use an xpath or make the css selector be chosen dynamically to make it futureproof. 

## **We will go with the latter**

**Comments**

In [13]:
for css_selector in comment_counter.attrs:
    if css_selector.startswith('data-v-'):
        comment_css_selector=css_selector
        break
    else:
        print("No CSS selector found for comments")
        
print("The CSS selector for the amount of comments is:",comment_css_selector)

The CSS selector for the amount of comments is: data-v-c10ff764


In [14]:
#perfect, we have the css selector, let's make it dynamic now
#comment_counter=soup.find(class_="comment-counter")
#if comment_counter.startswith('data-v-'):
#   comment_css_selector=comment_counter

### this doesn't work because i'm calling the ELEMENT, not the attributes of that element
#it's like you wanted to see what characteristics a person has, but you only got that person
#like if we asked a question "what hair does Peter have?" and the answer was: "That's Peter"

Now let's find the selectors for tags, upvotes and titles as well.

**Tags**

In [15]:
soup.find(class_="tag")

<li class="tag" data-v-0e7a0051=""><span data-v-0e7a0051="">#</span><div class="tooltip-slot" data-v-0e7a0051="" data-v-d8ca489a=""><span data-v-d8ca489a=""><span style="display: none;"><section class="popper tooltip-slot tag-actions guest" data-v-d8ca489a=""><!-- --></section> </span> <a class="" data-v-0e7a0051="" href="/tag/majtczak">majtczak</a></span></div></li>

In [16]:
#the css selector li class is the same as the one we want. This makes it easier.
tag_selector=soup.find(class_="tag")
tag_selector.attrs

{'data-v-0e7a0051': '', 'class': ['tag']}

In [17]:
#making a dynamic css selector for tags
for css_selector in tag_selector.attrs:
    if css_selector.startswith('data-v-'):
        tag_css_selector=css_selector
        break
    else:
        print("CSS selector for tags not found")

print("The CSS selector for Tags is:",tag_css_selector)

The CSS selector for Tags is: data-v-0e7a0051


**Upvotes**

In [18]:
soup.find(class_="dig")

<div class="dig" data-v-5bb34f93=""><p data-v-5bb34f93=""><span data-v-5bb34f93="">11524</span></p> <button data-v-5bb34f93="">Wykop</button></div>

In [19]:
upvote_selector=soup.find(class_="dig")
upvote_selector.attrs

{'data-v-5bb34f93': '', 'class': ['dig']}

In [20]:
for css_selector in upvote_selector.attrs:
    if css_selector.startswith('data-v-'):
        upvote_css_selector=css_selector
        break
    else:
        print("CSS selector for upvotes not found")
        
print("the CSS selector for upvotes is:",upvote_css_selector)


the CSS selector for upvotes is: data-v-5bb34f93


**Titles**

In [21]:
soup.find_all("a")

[<a class="logotype hybrid active" data-v-6c2d0fdd="" href="/" title="Wykop.pl">Wykop.pl</a>,
 <a class="logo hybrid active" data-v-6c2d0fdd="" href="/" title="Wykop.pl">Wykop.pl</a>,
 <a class="hybrid" data-v-6c2d0fdd="" href="/"><span data-v-6c2d0fdd="">Główna</span></a>,
 <a class="hybrid" data-v-6c2d0fdd="" href="/wykopalisko"><span data-v-6c2d0fdd="">Wykopalisko</span></a>,
 <a class="hybrid" data-v-6c2d0fdd="" href="/mikroblog"><span data-v-6c2d0fdd="">Mikroblog</span></a>,
 <a data-v-6c2d0fdd="" href="/logowanie">Zaloguj się</a>,
 <a data-v-6c2d0fdd="" href="/rejestracja">Zarejestruj się</a>,
 <a class="hybrid" data-v-5687662b="" href="/"><span data-v-5687662b="" style="color: rgb(67, 131, 175);"><em data-v-5687662b="">Wykop.pl</em></span></a>,
 <a class="hybrid" data-v-5687662b="" href="/k/ciekawostki"><span data-v-5687662b="" style="color: rgb(130, 73, 56);"><em data-v-5687662b="">Ciekawostki</em></span></a>,
 <a class="hybrid" data-v-5687662b="" href="/k/rozrywka"><span data-

This is a bit trickier, for example:

```python
<a class="" data-v-69e67e26="" href="/link/7353491/sebastian-m-wyszedl-na-wolnosc">
                             Sebastian M. wyszedł na wolność
                         </a>,
```                         
There is no "tag" or "div" or anything remotely unique to Titles. We can see that there is only "a" anchor tag that is very popular with other attributes, therefore we have to restrict our search even more. 

We can see that there is a href=/link/. Let's try with that.

We can get that using regex.

In [22]:
soup.find('a',href=re.compile(r'/link/'))

<a class="" data-v-69e67e26="" href="/link/7353491/sebastian-m-wyszedl-na-wolnosc">
                            Sebastian M. wyszedł na wolność
                        </a>

Perfect, now we can get the selector.
Since the selector is "connected" to both the link and the title, we will use regular expression to easily find the selector using the /link/ prefix

In [23]:
titles_selector=soup.find('a',href=re.compile(r'/link'))
titles_selector.attrs

{'data-v-69e67e26': '',
 'href': '/link/7353491/sebastian-m-wyszedl-na-wolnosc',
 'class': []}

In [24]:
for css_selector in titles_selector.attrs:
    if css_selector.startswith('data-v-'):
        titles_css_selector=css_selector
        break
    else:
        print("No CSS selector found for Titles")
        
print("The CSS selector for titles is:",titles_css_selector)
#Because of some unknown problem, this code will be added during the selenium scraping part

The CSS selector for titles is: data-v-69e67e26


# Articles information

**Upvote scores**

In [25]:
upvotes = []
for tag in soup.find_all('div',attrs={upvote_css_selector:True}): #the usage of 'div' to filter further is crucial(i checked)
    # despite using dedicated CSS selector, without 'div' element every result would be quadrupled. This is due to the fact that
    #(logically) the upvote_css_selector belongs to 4 elements with different tags (one of them is 'div')
    number_match = re.search(r"\d+", tag.text.strip())
    if number_match:
        upvotes.append(number_match.group())

print(upvotes)

print(len(upvotes))

['11524', '11033', '10846', '10406', '9716', '9051', '9116', '8727', '7644', '7532', '7068', '6972', '6771', '6796', '6711', '6735', '6677', '6638', '6554', '6483']
20


**Titles of the articles**

In [26]:

titles=[]
for h2 in soup.find_all('h2',attrs={titles_css_selector:True}): #if we only used titles_css_selector as an argument for 
                                                                #soup.find_all, we would get everything. We need to be 
                                                                #specific and use a filter, 'h2'
    titles_match=re.search(r".+",h2.text.strip())
    if titles_match:  # Check if there's an anchor tag within the h2
        titles.append(h2.text.strip())
 #print the extracted titles
print(titles)

['Sebastian M. wyszedł na wolność', 'Od prawie dwóch miesięcy Sebastianowi M. nadal nie postawiono zarzutów.', 'Śmiertelny wypadek na A1. Nadal nie zatrzymano uczestnika wypadku! WYKOP przypomina', 'Prokuratura umorzyła śledztwo w sprawie niezatrzymania Sebastiana M.', 'Śmierć mojej matki pracującej w Niemieckiej agencji pracy', 'Żona Sebastiana M. przerywa milczenie i pozywa WYKOP.pl', 'Najkrótsze Wiadomości w Historii!', 'Majtczak cdn.', 'Polscy żołnierze zakuci w kajdanki na białoruskiej granicy. W wojsku wrze', '315 km/h. Policja pozwoliła kierowcy BMW uciec do Emiratów', 'Strajk! W sobotę (31.08) nie włączaj wykopu', 'Janusz Rewiński nie żyje', 'Przemysław Babiarz zawieszony przez TVP za slowa o komunizmie', '#wykopefekt #afera Psy terroryzują gminę, patuska grozi', 'Były policjant zabił trzynastolatkę. Prokuratura i policjanci go chronią.', 'LIST GOŃCZY: Majtczak, Sebastian Waldemar (Sygnatura akt 3028 - 1. Ds 70.2023)', 'Właściciel wykopu to deweloper', 'Co z nim?? Nie zapominaj

**Tags**

In [27]:

hrefs = []
for a in soup.find_all(attrs={tag_css_selector:True}): #True because attrs is of class dictionary.
    try:
        href = a.attrs['href']  # Get the value of the 'href' attribute
        tag_match = re.match(r"/tag/([^/]+)$", href)  # Search for the tag name using regex
        if tag_match:  # If a match is found
            tag_name = tag_match.group(1)  # Extract the tag name from the matched group,
            #we use tag_match.group(1) so that regex will only the tag from the html syntax

            hrefs.append(tag_name)  # Append the extracted tag name to the list
    except:
        continue

print(hrefs)
print(len(hrefs))

['majtczak', 'wypadek', 'bmw', 'stopcham', 'motoryzacja', 'prawo', 'majtczak', 'bmw', 'morderstwo', 'a1', 'panstwozdykty', 'bezkarnosc', 'a1wypadek', 'sebastianmajtczak', 'zabojstwo', 'prokuratura', 'sledztwo', 'bmw', 'polska', 'polskiedrogi', 'majtczak', 'sebastianmajtczak', 'codziennymajtczak', 'nicsieniestalo', 'niemcy', 'januszebiznesu', 'afera', 'wykopefekt', 'pomocy', 'polskiedrogi', 'wypadek', 'samochody', 'wykop', 'afera', 'p0lka', 'wiadomosci', 'tvp', 'telewizja', 'media', 'majtczak', 'bmw', 'bialorus', 'granica', 'kryminalne', 'policja', 'przestepczosc', 'wydarzenia', 'kryminalistyka', 'prawo', 'wykop', 'wykopefekt', 'polska', 'afera', 'krecimymalysza', 'lecimywgorace', 'film', 'wydarzenia', 'ciekawostki', 'rewinski', 'siara', 'paryz2024', 'swiat', 'europa', 'sport', 'wykopefekt', 'afera', 'patologiazewsi', 'jedlanka', 'policja', 'policja', 'policjant', 'leszno', 'ziobro', 'polska', 'listgonczy', 'kryminalne', 'przestepczosc', 'policja', 'majtczak', 'sebastianmajtczak', 'nier

**Comment counter**

In [28]:
# when looking at the parsed html we can see that comments have <a class="comment-counter"
comments = []
for comment in soup.find_all(attrs={comment_css_selector:True}):
    comments_match = re.search(r"\d+", comment.text.strip())
    if comments_match:
        comments.append(comments_match.group())

print(comments)
print(len(comments))

['796', '301', '357', '578', '646', '1429', '1237', '355', '1135', '290', '541', '501', '907', '692', '265', '214', '305', '208', '806', '852']
20


To give us a sneak peek into what topics we are going to deal with, let's check some tags on the first page

In [29]:
html1 = requests.get(url)
bs =  BeautifulSoup(html1.text,  'html.parser')
print(len(bs.find_all(lambda tag: "polska" in tag.get_text())))

55


In [30]:
print(len(bs.find_all(lambda tag: "konfederacja" in tag.get_text())))

0


In [31]:
print(len(bs.find_all(lambda tag: "ekologia" in tag.get_text())))

0


In [32]:
print(len(bs.find_all(lambda tag: "policja" in tag.get_text())))

58


In [33]:
print(len(bs.find_all(lambda tag: "smierc" in tag.get_text())))

0


# Let's get to scraping

In [55]:
website = "https://wykop.pl/hity/roku/strona/1"
firefoxpath = GeckoDriverManager().install()
service_firefox = Service(executable_path=firefoxpath)
options_firefox = Options()
driver_firefox = webdriver.Firefox(service=service_firefox, options=options_firefox)  # Case sensitive .firefox won't work
                                                                                   # opens firefox

driver_firefox.maximize_window() # mazimizes browser's window
driver_firefox.get(website) # opens a website

time.sleep(5)

cookies_button_xpath ='/html/body/div[2]/div[1]/div[2]/div/div[2]/button[2]' #relative xpath for accepting cookies
content = driver_firefox.find_element('xpath',cookies_button_xpath) # finds the button for accepting cookies
content.click() # clicks the button

time.sleep(3) #increase if needed, it was enough




try:
    # Wait for the close button of the ad to be visible
    ad_xpath='/html/body/section/div[2]/main/section[2]/section/button'
    content = driver_firefox.find_element('xpath',ad_xpath) # finds the "close ad" button
    #if the ad apperas
    content.click() # clicks the "close ad" button 
except:
    # If the ad doesn't appear
    pass
time.sleep(5)


#"Hey, you already added this chunk of code, why did you add it again?"
#Great question! For some GODDAMN unknown reason, the CSS selector for titles
#is different when using requests and soup compared to 
#using selenium and soup (like right now)
#I checked it by hand and the second one (using selenium and soup) is correct
#adding this chunk of code here again fixed the issue
#Otherwise, the loop wouldn't have found any titles

#since there is only 20 news per page and we want about a 1000, let's do 50 pages. We need a loop for that
start = time.time()
all_upvotes = []
all_comments = []
all_hrefs = []
all_titles = []

for scrap_pages in range(1, 20):
    time.sleep(2)

    driver_firefox.get("https://wykop.pl/hity/roku/strona/"+str(scrap_pages))

    # in order to get the new content from new pages, we need to update the page's html every time it goes to the new page
    #thats why its included in the loop
    driver_firefox.execute_script("window.scrollTo(0, document.body.scrollHeight);") #ok for some reason this helped,
    #I understand the theory but instead of scraping 15 out of 20 (all posts that load the moment the page loads)
    #it scraped 0 out of 20 giving me empty lists, not sure why this fixes the problem but hey - it works
    new_page_source = driver_firefox.page_source

    soup = BeautifulSoup(new_page_source, 'html.parser')
    titles_selector=soup.find('a',href=re.compile(r'/link'))
    titles_selector.attrs
    for css_selector in titles_selector.attrs:
        if css_selector.startswith('data-v-'):
            titles_css_selector=css_selector
    # now, scrape data from the updated soup object for upvotes
    #### UPVOTE SCRAPING ####
    upvotes = []
    for tag in soup.find_all('div',attrs={upvote_css_selector:True}):
        number_match = re.search(r"\d+", tag.text.strip())
        if number_match:
            upvotes.append(number_match.group())
    # append upvotes data for the current page to the list
    all_upvotes.append(upvotes)

    # now, scrape data from the updated soup object for comments
    #### No. of comments scraping
    comments = []
    for comment in soup.find_all(attrs={comment_css_selector:True}):
        comments_match = re.search(r"\d+", comment.text.strip())
        if comments_match:
            comments.append(comments_match.group())

    # append comments data for the current page to the list
    all_comments.append(comments)
    #### TAG SCRAPING ####
    hrefs = []
    for a in soup.find_all(attrs={tag_css_selector:True}):
        try:
            href = a.attrs['href']  # Get the value of the 'href' attribute
            tag_match = re.search(r"/tag/([^/]+)$", href)  # Search for the tag name using regex
            if tag_match:  # If a match is found
                tag_name = tag_match.group(1)  # Extract the tag name from the matched group 
                hrefs.append(tag_name)  # Append the extracted tag name to the list
        except:
            continue

    all_hrefs.append(hrefs)

    #### TITLE SCRAPING ####
    titles=[]
    for h2 in soup.find_all('h2',attrs={titles_css_selector:True}): 
        titles_match=re.search(r".+",h2.text.strip())
        if titles_match:
            titles.append(h2.text.strip())
    all_titles.append(titles)


driver_firefox.close()


end = time.time()
print("how long the scraping took in seconds:",end-start)

how long the scraping took in seconds: 54.237539529800415


Let's make a dataframe out of what we got

In [56]:
DiscussionStats=pd.DataFrame({'Upvotes':all_upvotes,'Comments':all_comments,'Titles':all_titles})
DiscussionStats

,Upvotes,Comments,Titles
0,"[11524, 11033, 10846, 10406, 9716, 9051, 9116,...","[796, 301, 357, 578, 646, 1429, 1237, 355, 113...","[Sebastian M. wyszedł na wolność, Od prawie dw..."
1,"[6453, 6393, 6374, 6321, 6388, 6161, 6122, 616...","[322, 425, 1007, 391, 547, 641, 854, 1299, 457...",[Za 0.007g marihuany nieletni idzie do aresztu...
2,"[5557, 5405, 5331, 5305, 5261, 5190, 5176, 512...","[253, 347, 357, 481, 571, 672, 509, 325, 196, ...",[Wniosek formalny o wysłanie zespołu Figo Fago...
3,"[4970, 4881, 4817, 4804, 4776, 4815, 4723, 476...","[673, 263, 321, 171, 218, 579, 241, 276, 355, ...","[Morderca Łukasz Ż. zatrzymany w Niemczech, Ka..."
4,"[4527, 4697, 4515, 4555, 4515, 4464, 4435, 443...","[658, 526, 320, 156, 344, 368, 589, 177, 310, ...",[Zostałem oszukany na sporą sumę. Policja umar...
5,[],[],[]
6,"[4043, 3958, 3919, 3948, 3970, 3913, 3873, 392...","[282, 613, 414, 564, 283, 292, 347, 418, 337, ...",[Jak Fundacja Batorego (ta od Sorosa i Holland...
7,"[3750, 3717, 3730, 3770, 3742, 3708, 3703, 370...","[465, 150, 396, 645, 122, 236, 158, 263, 330, ...",[SpaceX Starship - udany test. Mechazilla dzia...
8,"[3620, 3629, 3591, 3614, 3598, 3547, 3718, 354...","[331, 453, 185, 302, 262, 134, 714, 656, 345, ...","[Ceny gazu na giełdach niskie, a my przepłacam..."
9,"[3536, 3425, 3436, 3414, 3412, 3421, 3405, 342...","[463, 619, 213, 197, 608, 380, 201, 308, 416, ...","[Chwila, w której uświadamiasz wyborcę PiS, że..."


Since we have list of lists, let's expand them so it looks more clear

In [49]:
#we'll use the pandas explode function
DiscussionStats_expanded = DiscussionStats.apply(pd.Series.explode, ignore_index=True)

print(DiscussionStats_expanded)

    Upvotes Comments                                             Titles
0     11524      796                    Sebastian M. wyszedł na wolność
1     11033      301  Od prawie dwóch miesięcy Sebastianowi M. nadal...
2     10846      357  Śmiertelny wypadek na A1. Nadal nie zatrzymano...
3     10406      578  Prokuratura umorzyła śledztwo w sprawie niezat...
4      9716      646  Śmierć mojej matki pracującej w Niemieckiej ag...
..      ...      ...                                                ...
956    1516      203  Co wie były Prezes Urzędu Ochrony Danych Osobo...
957    1512      177  Śledztwo w sprawie śmierci Jolanty Brzeskiej u...
958    1527      203  Zandberg: Apelujemy do premiera i KO o wycofan...
959    1524       93  TVP na żywo zasłonił wyniki głosowania nad TVP...
960    1511      267            MBank cieszy się z podwyżki cen za wodę

[961 rows x 3 columns]


In [52]:
#Tags is a list of list, we can't use explode so we will use list comprehension
Tags=all_hrefs.copy()
Tags=[tag for unlist in Tags for tag in unlist]
print(Tags[:20])
len(Tags)

['majtczak', 'wypadek', 'bmw', 'stopcham', 'motoryzacja', 'prawo', 'majtczak', 'bmw', 'morderstwo', 'a1', 'panstwozdykty', 'bezkarnosc', 'a1wypadek', 'sebastianmajtczak', 'zabojstwo', 'prokuratura', 'sledztwo', 'bmw', 'polska', 'polskiedrogi']


4463

As we can see the amount of tags is 4463 (as of writing) compared to 1000 rows of the DataFrame (as of writing)

Due to the way the site is structured this is the only solution, to have tags and the rest separate

In [51]:
print(DiscussionStats_expanded.head(20))

   Upvotes Comments                                             Titles
0    11524      796                    Sebastian M. wyszedł na wolność
1    11033      301  Od prawie dwóch miesięcy Sebastianowi M. nadal...
2    10846      357  Śmiertelny wypadek na A1. Nadal nie zatrzymano...
3    10406      578  Prokuratura umorzyła śledztwo w sprawie niezat...
4     9716      646  Śmierć mojej matki pracującej w Niemieckiej ag...
5     9051     1429  Żona Sebastiana M. przerywa milczenie i pozywa...
6     9116     1237                  Najkrótsze Wiadomości w Historii!
7     8727      355                                      Majtczak cdn.
8     7644     1135  Polscy żołnierze zakuci w kajdanki na białorus...
9     7532      290  315 km/h. Policja pozwoliła kierowcy BMW uciec...
10    7068      541        Strajk! W sobotę (31.08) nie włączaj wykopu
11    6972      501                           Janusz Rewiński nie żyje
12    6771      907  Przemysław Babiarz zawieszony przez TVP za slo...
13    

Since tags is not a dataframe but a list, we can't use anything like .head() to show the most popular tags

We can use Counter from collections library

In [39]:
from collections import Counter

count_tags = Counter(Tags)

# get the 5 most common tags
most_common_tags = count_tags.most_common(5) #change (5) to whatever number you want if you want more. (5) indicates 
#5 most common

print(most_common_tags)

[('polska', 45), ('afera', 19), ('majtczak', 17), ('wypadek', 14), ('prawo', 13)]
